In [1]:
import pandas as pd
import numpy as np

#1= positive; 0: negetive; -1: uncertain
trainDF = pd.read_csv('CheXpert-v1.0-small/train.csv', dtype=str)
trainDF = trainDF.fillna(value = '0.0')

# 223414 entries in CSV

In [2]:
## Number of unique patients in train set
s = trainDF['Path']
x =(s.str.rsplit('/', 3, expand = True))
x.head()
#len(x[1].unique())

,0,1,2,3
0,CheXpert-v1.0-small/train,patient00001,study1,view1_frontal.jpg
1,CheXpert-v1.0-small/train,patient00002,study2,view1_frontal.jpg
2,CheXpert-v1.0-small/train,patient00002,study1,view1_frontal.jpg
3,CheXpert-v1.0-small/train,patient00002,study1,view2_lateral.jpg
4,CheXpert-v1.0-small/train,patient00003,study1,view1_frontal.jpg


In [3]:
# #removing frontal image entries
# indexFrontal = trainDF[trainDF['Frontal/Lateral'] == 'Frontal' ].index
# trainDF = trainDF.drop(indexFrontal)

In [4]:
# Total number of lateral files.
trainDF[trainDF['Frontal/Lateral'] == 'Lateral']['Path'].count()

32387

In [5]:
#32387 lateral
#33151 frontal lateral pairs generated
# IDEA: 
# Loop over trainDF as f,
#      if f is lateral:
#          get folder path of f and
#          if it is in list 'visited', continue
#          else add filder to list visited
#          and copy file to other location (as lateral.jpg), add to newDF
#          check for other files (this will be frontal), copy and add to newDF.

In [79]:
from shutil import copyfile
import glob
import os
from tqdm.notebook import trange, tqdm
tqdm.pandas()

from pathlib import Path
from shutil import copyfile

newDataDF = pd.DataFrame(columns = ['Path', 'No_Finding', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural_Effusion'])

src_directory = ""
dest_directory = "pair-wise/"
visitedList = []
l = []
for index, row in tqdm(trainDF.iterrows()):
    if row["Frontal/Lateral"] == 'Lateral':
        s = row['Path']
        x =(s.rsplit('/', 1))
        
        '''check if folder already visited'''
        if x[0] in visitedList: #x[0] is the folder name, x[1] has the file name.
            continue
        else:
            visitedList.append(x[0])
        
        '''filesinFolder: list of all jpg files in current folder'''
        filesinFolder = glob.glob(os.path.join(x[0], '*.jpg')) 
        
#         #Print folders with single lateral files
#         if len(filesinFolder) == 1:
#             print(filesinFolder)

        '''Continue if there is 1 file in current folder.'''
        if len(filesinFolder) == 1:
            continue
            
        '''Check if all images in folder are of same type. Continue, if so.'''
        f_l = []
        for f in filesinFolder:
            f_l.append((trainDF[trainDF['Path'] == f]['Frontal/Lateral']).item())
        # Continue if items in f_l are all same, i.e., all 'frontal' or all 'lateral'. 
        if f_l.count(f_l[0]) == len(f_l):
            #print(filesinFolder[0])
            continue
        
#         #Check how many files files are in each folder
#         if len(filesinFolder) not in l:
#             l.append(len(filesinFolder))
#             print("Added: ", len(filesinFolder), "at index ",index,"\n")
        
        '''Transfer files to new folder, and create new dataframe.'''
        # Case 1: Folder has 1 lateral and 1 frontal images.
        if len(filesinFolder) == 2:
            # Create one folder.
            temp = row['Path'].rsplit('/', 3) #temp list to store loc, patientID, studyNo, filename
            new_name = temp[1]+'-'+temp[2]
            curr_dest = Path(os.path.join(dest_directory, new_name))
            (curr_dest).mkdir(parents=True, exist_ok=True)
            #Copy files.
            for f1 in filesinFolder: # OR loop over f_l to get file name.
                dest_file = os.path.join(curr_dest, ((trainDF[trainDF["Path"] == f1]['Frontal/Lateral']).item() + ".jpg"))
                copyfile(f1, dest_file)
            #Update new dataframe
            # Loc: curr_dest, Diseases: trainDF[trainDF["Path"] == filesinFolder[0]]["MANIFESTATIONS"]
            dict_curr_dest = {
                              "Path": curr_dest,
                              "No_Finding": trainDF[trainDF["Path"] == filesinFolder[0]]["No Finding"].item(),
                              "Atelectasis": trainDF[trainDF["Path"] == filesinFolder[0]]["Atelectasis"].item(),
                              "Cardiomegaly": trainDF[trainDF["Path"] == filesinFolder[0]]["Cardiomegaly"].item(),
                              "Consolidation": trainDF[trainDF["Path"] == filesinFolder[0]]["Consolidation"].item(),
                              "Edema": trainDF[trainDF["Path"] == filesinFolder[0]]["Edema"].item(),
                              "Pleural_Effusion": trainDF[trainDF["Path"] == filesinFolder[0]]["Pleural Effusion"].item(),
                            }

            newDataDF = newDataDF.append(dict_curr_dest, ignore_index=True)
            
        elif len(filesinFolder) == 3:
            # Case 2: Folder has 2 Lateral, 1 frontal images.
            if f_l.count("Lateral") == 2:
                
                idx_frontal = f_l.index("Frontal")
                #print("idx_frontal", idx_frontal)
                idx_lateral = [0, 1, 2]
                idx_lateral.remove(idx_frontal)
                #print("idx_lateral", idx_lateral)
                for i in idx_lateral:
                    #Create folder, with suffix "-i"
                    temp = row['Path'].rsplit('/', 3) #temp list to store loc, patientID, studyNo, filename
                    new_name = temp[1]+'-'+temp[2]+'-L-'+str(i)
                    curr_dest = Path(os.path.join(dest_directory, new_name))
                    (curr_dest).mkdir(parents=True, exist_ok=True)
                    #Copy files.
                    dest_file_f = os.path.join(curr_dest, "Frontal.jpg")
                    copyfile(filesinFolder[idx_frontal], dest_file_f)
                    dest_file_l = os.path.join(curr_dest, "Lateral.jpg")
                    copyfile(filesinFolder[i], dest_file_l)
                    #ToDo: Update dataframe
                    # Loc: curr_dest, Diseases: trainDF[trainDF["Path"] == filesinFolder[i]]["MANIFESTATIONS"]
                    dict_curr_dest = {
                                      "Path": curr_dest,
                                      "No_Finding": trainDF[trainDF["Path"] == filesinFolder[i]]["No Finding"].item(),
                                      "Atelectasis": trainDF[trainDF["Path"] == filesinFolder[i]]["Atelectasis"].item(),
                                      "Cardiomegaly": trainDF[trainDF["Path"] == filesinFolder[i]]["Cardiomegaly"].item(),
                                      "Consolidation": trainDF[trainDF["Path"] == filesinFolder[i]]["Consolidation"].item(),
                                      "Edema": trainDF[trainDF["Path"] == filesinFolder[i]]["Edema"].item(),
                                      "Pleural_Effusion": trainDF[trainDF["Path"] == filesinFolder[i]]["Pleural Effusion"].item(),
                                    }
                    
                    newDataDF = newDataDF.append(dict_curr_dest, ignore_index=True)
                    
            # Case 3: Folder has 1 Lateral, 2 frontal images.
            elif f_l.count("Frontal") == 2:
                idx_lateral = f_l.index("Lateral")
                idx_frontal = [0, 1, 2]
                idx_frontal.remove(idx_lateral)
                for i in idx_frontal:
                    #Create folder, with suffix "-i"
                    temp = row['Path'].rsplit('/', 3) #temp list to store loc, patientID, studyNo, filename
                    new_name = temp[1]+'-'+temp[2]+'-F-'+str(i)
                    curr_dest = Path(os.path.join(dest_directory, new_name))
                    (curr_dest).mkdir(parents=True, exist_ok=True)
                    #Copy files.
                    dest_file_l = os.path.join(curr_dest, "Lateral.jpg")
                    copyfile(filesinFolder[idx_lateral], dest_file_l)
                    dest_file_f = os.path.join(curr_dest, "Frontal.jpg")
                    copyfile(filesinFolder[i], dest_file_f)
                    #Update dataframe
                    # Loc: curr_dest, Diseases: trainDF[trainDF["Path"] == filesinFolder[i]]["MANIFESTATIONS"]
                    dict_curr_dest = {
                                      "Path": curr_dest,
                                      "No_Finding": trainDF[trainDF["Path"] == filesinFolder[i]]["No Finding"].item(),
                                      "Atelectasis": trainDF[trainDF["Path"] == filesinFolder[i]]["Atelectasis"].item(),
                                      "Cardiomegaly": trainDF[trainDF["Path"] == filesinFolder[i]]["Cardiomegaly"].item(),
                                      "Consolidation": trainDF[trainDF["Path"] == filesinFolder[i]]["Consolidation"].item(),
                                      "Edema": trainDF[trainDF["Path"] == filesinFolder[i]]["Edema"].item(),
                                      "Pleural_Effusion": trainDF[trainDF["Path"] == filesinFolder[i]]["Pleural Effusion"].item(),
                                    }
                    
                    newDataDF = newDataDF.append(dict_curr_dest, ignore_index=True)
                    
        
    #src_file = row['Path']
    #dest_file = dest_directory + "/" + row['new_filename']

    #copyfile(src_file, dest_file)

In [80]:
newDataDF

,Path,No_Finding,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural_Effusion
0,pair-wise/patient00002-study1,0.0,0.0,0.0,-1.0,0.0,0.0
1,pair-wise/patient00004-study1,1.0,0.0,0.0,0.0,0.0,0.0
2,pair-wise/patient00005-study1,1.0,0.0,0.0,0.0,0.0,0.0
3,pair-wise/patient00009-study1,0.0,1.0,1.0,0.0,0.0,0.0
4,pair-wise/patient00010-study1,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
33146,pair-wise/patient60704-study1-L-0,0.0,0.0,-1.0,0.0,0.0,0.0
33147,pair-wise/patient60704-study1-L-2,0.0,0.0,-1.0,0.0,0.0,0.0
33148,pair-wise/patient60706-study1,0.0,0.0,1.0,0.0,0.0,0.0
33149,pair-wise/patient60707-study1,0.0,0.0,1.0,0.0,0.0,0.0


In [81]:
newDataDF.describe()

,Path,No_Finding,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural_Effusion
count,33151,33151,33151,33151,33151,33151,33151
unique,33151,2,3,3,3,3,3
top,pair-wise/patient11270-study17,0.0,0.0,0.0,0.0,0.0,0.0
freq,1,27625,25456,28049,27870,29341,21520


In [82]:
newDataDF.to_csv("pair-wise.csv", index=False)

In [51]:

dest_directory = "pair-wise/"
from pathlib import Path
curr_dest = Path(os.path.join(dest_directory, 'my/dir2'))
print((curr_dest).mkdir(parents=True, exist_ok=True))

None


In [30]:
# List index of list with value 3.
lx = [i for i, j in enumerate(l) if j == 3]
# from collections import Counter
# Counter(l)
lx

[]

In [36]:
trainDF.iloc[762]['Path']

'CheXpert-v1.0-small/train/patient00183/study3/view1_frontal.jpg'

In [8]:
trainDF[trainDF['Path']=='CheXpert-v1.0-small/train/patient00117/study1/view2_frontal.jpg']['Frontal/Lateral']

422    Frontal
Name: Frontal/Lateral, dtype: object

In [34]:
import glob
d = trainDF['Path'][0]
print(d)

for file in glob.glob(d+'/*.*'):
    print(file)

CheXpert-v1.0-small/train/patient00001/study1/view1_frontal.jpg


In [1]:
newDataDF.describe()

NameError: name 'newDataDF' is not defined